<p style="text-align:center">
    <a>
    <img src="rain.jpg" width="800" alt="Skills Network Logo">
    </a>
</p>

<h1 align="center"><font size="5">Progetto Finale: Predizione delle piogge in Australia</font></h1>


### Introduzione

Di seguito, utilizzeremo gli algoritmi di classificazione per creare un modello basato sui dati di addestramento e per valutare i dati di test utilizzando le metriche di valutazione apprese nel corso.

Utilizzeremo alcuni degli algoritmi più famosi, in particolare:

1. Regressione lineare
2. KNN
3. Decision Trees
4. Regressione logistica
5. SVM

Valuteremo i nostri utilizzando:

1.  Accuracy Score
2.  Indice di Jaccard
3.  F1-Score
4.  LogLoss
5.  Errore assoluto medio
6.  Errore quadratico medio
7.  R2

### Fonti dei dati

La fonte originale dei dati è l'Australian Government's Bureau of Meteorology e i dati più recenti possono essere raccolti da [http://www.bom.gov.au/climate/dwo/](http://www.bom.gov.au/climate/dwo/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01).

Il set di dati da utilizzare ha colonne extra come "RainToday" e il nostro obiettivo è "RainTomorrow", che è stato raccolto [https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData](https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)

Il dataset contiene osservazioni giornaliere dal 2008 al 20147. I campi inclusi sono i seguenti:

| Campo         | Descrizione                                            | Unità           | Tipo   |
| ------------- | ------------------------------------------------------ | --------------- | ------ |
| Date          | Data dell'osservazione YYYY-MM-DD                      | Data            | object |
| Location      | Luogo dell'osservazione                                | Luogo           | object |
| MinTemp       | Temperatura minima                                     | Celsius         | float  |
| MaxTemp       | Temperatura massima                                    | Celsius         | float  |
| Rainfall      | Quantità di pioggia                                    | Millimeters     | float  |
| Evaporation   | Quantità di evaporazione                               | Millimeters     | float  |
| Sunshine      | Quantità di ore soleggiate                             | hours           | float  |
| WindGustDir   | Direzione della raffica più forte                      | Compass Points  | object |
| WindGustSpeed | Velocità della raffica più forte                       | Kilometers/Hour | object |
| WindDir9am    | Direzione del vento mediata su 10 minuti prima delle 9 | Compass Points  | object |
| WindDir3pm    | Direzione del vento mediata su 10 minuti prima delle 15| Compass Points  | object |
| WindSpeed9am  | Velocità del vento mediata su 10 minuti prima delle 9  | Kilometers/Hour | float  |
| WindSpeed3pm  | Velocità del vento mediata su 10 minuti prima delle 15 | Kilometers/Hour | float  |
| Humidity9am   | Umidità alle 9                                         | Percent         | float  |
| Humidity3pm   | Umidità alle 15                                        | Percent         | float  |
| Pressure9am   | Pressione atmosferica sul livello del mare alle 9      | Hectopascal     | float  |
| Pressure3pm   | Pressione atmosferica sul livello del mare alle 15     | Hectopascal     | float  |
| Cloud9am      | Porzione del cielo oscurata dalle nuvole alle 9        | Eights          | float  |
| Cloud3pm      | Porzione del cielo oscurata dalle nuvole alle 15       | Eights          | float  |
| Temp9am       | Temperatura alle 9                                     | Celsius         | float  |
| Temp3pm       | Temperatura alle 15                                    | Celsius         | float  |
| RainToday     | Presenza di pioggia oggi                               | Yes/No          | object |
| RainTomorrow  | Presenza di pioggia domani                             | Yes/No          | float  |

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score, f1_score, log_loss, confusion_matrix, accuracy_score
import sklearn.metrics as metrics

### Importazione dei dati


In [2]:
df = pd.read_csv("Weather_Data.csv")
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


### Data Preprocessing


#### One Hot Encoding


Convertiamo le variabili categoriche in binarie:


In [3]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

Modifichiamo la colonna ***RainTomorrow*** sostituendo Yes/No con 1/0.

In [4]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

### Separare le X dalla y

In [5]:
df_sydney_processed.drop('Date',axis=1,inplace=True) #non ci serve

In [6]:
df_sydney_processed = df_sydney_processed.astype(float)

In [7]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

### Regressione Lineare

#### Task 1) Usando la funzione `train_test_split` separiamo il dataset in train e test.

In [8]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=10)

#### Task 2) Creiamo e addestriamo un modello di regressione lineare usando il training data.


In [9]:
LinearReg = LinearRegression()
LinearReg.fit(x_train,y_train)

LinearRegression()

#### Task 3) Facciamo una predizione utilizzando il test data.

In [10]:
predictions = LinearReg.predict(x_test)

#### Task 4) Calcoliamo MAE, MSE e R2.

In [11]:
LinearRegression_MAE = np.mean(np.absolute(y_test-predictions))
LinearRegression_MSE = np.mean(np.absolute((y_test-predictions)**2))
LinearRegression_R2 = 1-(np.sum(np.absolute(predictions-y_test))/np.sum(np.absolute(y_test-np.mean(y_test))))

#### Task 5) Mostriamo i risultati in una nuova tabella.


In [12]:
Report = pd.DataFrame({'MAE':[LinearRegression_MAE], 'MSE':[LinearRegression_MSE], 'R2':[LinearRegression_R2]})
Report.index = ['Linear Regression Parameters']
Report

,MAE,MSE,R2
Linear Regression Parameters,0.256323,0.115717,0.365544


### KNN

#### Q6) Creiamo e addestriamo un modello KNN usando 4 neighbors come iperparametro.

In [13]:
KNN = KNeighborsClassifier(n_neighbors=4)
KNN.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=4)

#### Q7) Prediciamo i risultati usando il test data.

In [14]:
predictions = KNN.predict(x_test)

#### Q8) Calcoliamo l'accuracy, l'indice di Jaccard e l'F1-score.

In [15]:
KNN_Accuracy_Score = accuracy_score(y_test, predictions)
KNN_JaccardIndex = jaccard_score(y_test, predictions)
KNN_F1_Score = f1_score(y_test, predictions)

print(f'''
Accuracy Score: {KNN_Accuracy_Score}
Jaccard Index: {KNN_JaccardIndex}
F1 Score: {KNN_F1_Score}''')


Accuracy Score: 0.8183206106870229
Jaccard Index: 0.4251207729468599
F1 Score: 0.5966101694915255


### Decision Tree


#### Task 9) Creiamo un Decision Tree.

In [16]:
Tree = DecisionTreeClassifier()
Tree.fit(x_train, y_train)

DecisionTreeClassifier()

#### Q10) Prediciamo i risultati usando il test data.

In [17]:
predictions = Tree.predict(x_test)

#### Q11) Calcoliamo ancora accuracy, indice di Jaccard e F1-Score.

In [18]:
Tree_Accuracy_Score = accuracy_score(y_test, predictions)
Tree_JaccardIndex = jaccard_score(y_test, predictions)
Tree_F1_Score = f1_score(y_test, predictions)

print(f'''
Accuracy Score: {Tree_Accuracy_Score}
Jaccard Index: {Tree_JaccardIndex}
F1 Score: {Tree_F1_Score}''')


Accuracy Score: 0.76793893129771
Jaccard Index: 0.4153846153846154
F1 Score: 0.5869565217391305


### Regressione Logistica

#### Task 12) Creiamo un modello di Regressione Logistica usando il parametro `solver` settato su `liblinear`. 

In [19]:
LR = LogisticRegression(solver='liblinear')
LR.fit(x_train, y_train)

LogisticRegression(solver='liblinear')

#### Task 13) Prediciamo i risultati della classificazione e anche le probabilità assegnate a ciascun campione. 

In [20]:
predictions = LR.predict(x_test)

In [21]:
predict_proba = LR.predict_proba(x_test)

#### Task 14) Calcoliamo le stesse metriche usate sopra, aggiungendo anche il Log-Loss.

In [22]:
LR_Accuracy_Score = accuracy_score(y_test, predictions)
LR_JaccardIndex = jaccard_score(y_test, predictions)
LR_F1_Score = f1_score(y_test, predictions)
LR_Log_Loss = log_loss(y_test, predict_proba)

print(f'''
Accuracy Score: {LR_Accuracy_Score}
Jaccard Index: {LR_JaccardIndex}
F1 Score: {LR_F1_Score}
Log Loss: {LR_Log_Loss}''')


Accuracy Score: 0.8366412213740458
Jaccard Index: 0.5158371040723982
F1 Score: 0.6805970149253731
Log Loss: 0.35699291661803045


### SVM


#### Task 15) Creaiamo un modello di Support Vector Machine.

In [23]:
SVM = svm.SVC(kernel='linear')
SVM.fit(x_train, y_train)

SVC(kernel='linear')

#### Q16) Prediciamo i risultati usando il test data.

In [24]:
predictions = SVM.predict(x_test)

#### Q17) Calcoliamo le metriche appropriate.

In [25]:
SVM_Accuracy_Score = accuracy_score(y_test, predictions)
SVM_JaccardIndex = jaccard_score(y_test, predictions, pos_label=0)
SVM_F1_Score = f1_score(y_test, predictions)

print(f'''
Accuracy Score: {SVM_Accuracy_Score}
Jaccard Index: {SVM_JaccardIndex}
F1 Score: {SVM_F1_Score}''')


Accuracy Score: 0.8381679389312977
Jaccard Index: 0.8044280442804428
F1 Score: 0.6807228915662651


### Report


#### Q18) Mostriamo tutte le metriche calcolate in un'unica tabella per poterle confrontare.

\*Il Log-Loss è solo per il modello di Regressione Logistica

In [26]:
Accuracy = [KNN_Accuracy_Score, Tree_Accuracy_Score, LR_Accuracy_Score, SVM_Accuracy_Score]
Jaccard = [KNN_JaccardIndex, Tree_JaccardIndex, LR_JaccardIndex, SVM_JaccardIndex]
F1 = [KNN_F1_Score, Tree_F1_Score, LR_F1_Score, SVM_F1_Score]
LogLoss = [np.nan, np.nan, LR_Log_Loss, np.nan]
Report = pd.DataFrame({'Accuracy Score':Accuracy, 'Jaccard Index':Jaccard, 'F1 Score':F1, 'Log Loss':LogLoss})
Report.index = ['KNN', 'Decision Tree', 'Logistic Regression', 'SVM']
Report

,Accuracy Score,Jaccard Index,F1 Score,Log Loss
KNN,0.818321,0.425121,0.596610,NaN
Decision Tree,0.767939,0.415385,0.586957,NaN
Logistic Regression,0.836641,0.515837,0.680597,0.356993
SVM,0.838168,0.804428,0.680723,NaN
